In [104]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 36)
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.precision', 4)

# Prepare Dataset - Home Price

In [105]:
df_home_price = pd.read_csv("City_Zhvi_AllHomes.csv")

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
df_home_price.loc[:, "State"] = df_home_price["State"].map(states)


df_home_price = df_home_price.set_index(["State", "RegionName"])
df_home_price = df_home_price.loc[:, ["-" in col for col in df_home_price.columns]]


quarters = []
for year_month in df_home_price.columns:
    year = year_month[:4]
    month = year_month[-2:]
    if month in ["01", "02", "03"]:
        quarters.append("{}q1".format(year_month[:4]))
    elif month in ["04", "05", "06"]:
        quarters.append("{}q2".format(year_month[:4]))
    elif month in ["07", "08", "09"]:
        quarters.append("{}q3".format(year_month[:4]))
    elif month in ["10", "11", "12"]:
        quarters.append("{}q4".format(year_month[:4]))

df_home_price_quarter = df_home_price.T.groupby(quarters).mean().T
df_home_price_quarter = df_home_price_quarter.loc[:, df_home_price_quarter.columns >= "2000q1"]

df_home_price_quarter

                                       2000q1       2000q2       2000q3       2000q4       2001q1  ...       2015q3       2015q4       2016q1       2016q2    2016q3
State        RegionName                                                                            ...                                                              
New York     New York                     NaN          NaN          NaN          NaN          NaN  ...  557200.0000  572833.3333  582866.6667  591633.3333  587200.0
California   Los Angeles          207066.6667  214466.6667  220966.6667  226166.6667  233000.0000  ...  547266.6667  557733.3333  566033.3333  577466.6667  584050.0
Illinois     Chicago              138400.0000  143633.3333  147866.6667  152133.3333  156933.3333  ...  208300.0000  207900.0000  206066.6667  208200.0000  212000.0
Pennsylvania Philadelphia          53000.0000   53633.3333   54133.3333   54700.0000   55333.3333  ...  121233.3333  122200.0000  123433.3333  126933.3333  128700.0
Arizona   

# Prepare Dataset - University Town

In [106]:
df_uni_town = pd.read_csv('university_towns.txt', delimiter="\t", header=None, names=["data"])

df_uni_town.loc[:, "isState"] = df_uni_town["data"].str.strip().str.endswith("[edit]")
df_uni_town.loc[:, "State"] = np.where(df_uni_town["isState"], df_uni_town["data"], np.nan)
df_uni_town.loc[:, "State"] = df_uni_town["State"].fillna(method="ffill").str.replace("\[edit\]", "")

df_uni_town.loc[:, "RegionName"] = np.where(df_uni_town["isState"], np.nan, df_uni_town["data"])
df_uni_town.loc[:, "RegionName"] = df_uni_town["RegionName"].str.split(" \(").str[0]
df_uni_town = df_uni_town.dropna()

df_uni_town = df_uni_town[["State", "RegionName"]].reset_index(drop=True)

df_uni_town

         State     RegionName
0      Alabama         Auburn
1      Alabama       Florence
2      Alabama   Jacksonville
3      Alabama     Livingston
4      Alabama     Montevallo
5      Alabama           Troy
6      Alabama     Tuscaloosa
7      Alabama       Tuskegee
8       Alaska      Fairbanks
9      Arizona      Flagstaff
..         ...            ...
507  Wisconsin        Madison
508  Wisconsin      Menomonie
509  Wisconsin      Milwaukee
510  Wisconsin        Oshkosh
511  Wisconsin    Platteville
512  Wisconsin    River Falls
513  Wisconsin  Stevens Point
514  Wisconsin       Waukesha
515  Wisconsin     Whitewater
516    Wyoming        Laramie

[517 rows x 2 columns]

# Prepare Dataset - GDP Level

In [107]:
df_gdp = pd.read_excel(
    "gdplev.xls", sheet_name="Sheet1",
    usecols="E,G", names=["quarter", "gdp"],
    header=7
)

df_gdp.loc[:, "gdpDeclined"] = df_gdp["gdp"].diff() < 0

df_gdp.loc[:, "isRecession"] = False

for i in np.arange(len(df_gdp) - 2):
    
    if df_gdp.loc[i, "gdpDeclined"] and df_gdp.loc[i+1, "gdpDeclined"]:
        df_gdp.loc[i, "isRecession"] = True

        
    if (i != 0) and df_gdp.loc[i-1, "isRecession"] and not ((not df_gdp.loc[i, "gdpDeclined"]) and (not df_gdp.loc[i+1, "gdpDeclined"])):
        df_gdp.loc[i, "isRecession"] = True
        
        
df_gdp.loc[:, "recessionGroup"] = np.nan

recession_group_count = 0
for i in np.arange(1, len(df_gdp)):
    
    if (not df_gdp.loc[i-1, "isRecession"]) and df_gdp.loc[i, "isRecession"]:
        recession_group_count += 1
        df_gdp.loc[i, "recessionGroup"] = recession_group_count
    
    if df_gdp.loc[i , "isRecession"]:
        df_gdp.loc[i, "recessionGroup"] = recession_group_count

df_gdp

    quarter      gdp  gdpDeclined  isRecession  recessionGroup
0    1947q1   1934.5        False        False             NaN
1    1947q2   1932.3         True         True             1.0
2    1947q3   1930.3         True         True             1.0
3    1947q4   1960.7        False        False             NaN
4    1948q1   1989.5        False        False             NaN
5    1948q2   2021.9        False        False             NaN
6    1948q3   2033.2        False        False             NaN
7    1948q4   2035.3        False        False             NaN
8    1949q1   2007.5         True         True             2.0
9    1949q2   2000.8         True         True             2.0
..      ...      ...          ...          ...             ...
268  2014q1  15747.0         True        False             NaN
269  2014q2  15900.8        False        False             NaN
270  2014q3  16094.5        False        False             NaN
271  2014q4  16186.7        False        False         

In [108]:
def compute_recession_stats(df_group):
    result = pd.Series()
    result.loc["recessionStart"] = df_group["quarter"].values[0]
    result.loc["recessionEnd"] = df_group["quarter"].values[-1]
    result.loc["recessionBottom"] = df_group.sort_values(by="gdp")["quarter"].values[0]
    return result

df_gdp[df_gdp["isRecession"]].groupby("recessionGroup").apply(compute_recession_stats)

               recessionStart recessionEnd recessionBottom
recessionGroup                                            
1.0                    1947q2       1947q3          1947q3
2.0                    1949q1       1949q4          1949q2
3.0                    1953q3       1954q1          1954q1
4.0                    1957q4       1958q1          1958q1
5.0                    1969q4       1970q1          1970q1
6.0                    1974q3       1975q1          1975q1
7.0                    1980q2       1980q3          1980q3
8.0                    1981q4       1982q3          1982q1
9.0                    1990q4       1991q1          1991q1
10.0                   2008q3       2009q2          2009q2

# Run Tests

In [117]:
df_home_price_quarter.sort_index()

                               2000q1       2000q2       2000q3       2000q4       2001q1  ...       2015q4       2016q1       2016q2    2016q3                label
State     RegionName                                                                       ...                                                                      
Alabama   Adamsville       69033.3333   69166.6667   69800.0000   71966.6667   73466.6667  ...   73866.6667   74166.6667   74933.3333   74700.0  non-university town
          Alabaster       122133.3333  123066.6667  123166.6667  123700.0000  123233.3333  ...  151733.3333  153466.6667  155100.0000  155850.0  non-university town
          Albertville      73966.6667   72600.0000   72833.3333   74200.0000   75900.0000  ...   91366.6667   92000.0000   92466.6667   92200.0  non-university town
          Arab             83766.6667   81566.6667   81333.3333   82966.6667   84200.0000  ...  112233.3333  110033.3333  110100.0000  112000.0  non-university town
          

In [119]:
df_uni_town.loc[:, "label"] = "university town"
df_uni_town = df_uni_town.set_index(["State", "RegionName"])

df_uni_town

                                   label
State     RegionName                    
Alabama   Auburn         university town
          Florence       university town
          Jacksonville   university town
          Livingston     university town
          Montevallo     university town
          Troy           university town
          Tuscaloosa     university town
          Tuskegee       university town
Alaska    Fairbanks      university town
Arizona   Flagstaff      university town
...                                  ...
Wisconsin Madison        university town
          Menomonie      university town
          Milwaukee      university town
          Oshkosh        university town
          Platteville    university town
          River Falls    university town
          Stevens Point  university town
          Waukesha       university town
          Whitewater     university town
Wyoming   Laramie        university town

[517 rows x 1 columns]

In [120]:
df_home_price_quarter.index.isin(df_uni_town.index)

array([False, False, False, ..., False, False, False])

In [127]:

df_home_price_quarter.loc[:, "label"] = np.where(df_home_price_quarter.index.isin(df_uni_town.index), "university town", "non-university town")

df_home_price_quarter

                                       2000q1       2000q2       2000q3       2000q4       2001q1  ...       2015q4       2016q1       2016q2    2016q3                label
State        RegionName                                                                            ...                                                                      
New York     New York                     NaN          NaN          NaN          NaN          NaN  ...  572833.3333  582866.6667  591633.3333  587200.0  non-university town
California   Los Angeles          207066.6667  214466.6667  220966.6667  226166.6667  233000.0000  ...  557733.3333  566033.3333  577466.6667  584050.0  non-university town
Illinois     Chicago              138400.0000  143633.3333  147866.6667  152133.3333  156933.3333  ...  207900.0000  206066.6667  208200.0000  212000.0  non-university town
Pennsylvania Philadelphia          53000.0000   53633.3333   54133.3333   54700.0000   55333.3333  ...  122200.0000  123433.3333  12693

In [131]:
quarter_before_recession = "2008q2"
recession_bottom = "2009q2"

In [132]:
df_home_price_quarter.loc[:, "price ratio"] = df_home_price_quarter[quarter_before_recession] / df_home_price_quarter[recession_bottom]

df_home_price_quarter

                                       2000q1       2000q2       2000q3       2000q4       2001q1  ...       2016q1       2016q2    2016q3                label  price ratio
State        RegionName                                                                            ...                                                                      
New York     New York                     NaN          NaN          NaN          NaN          NaN  ...  582866.6667  591633.3333  587200.0  non-university town       1.0818
California   Los Angeles          207066.6667  214466.6667  220966.6667  226166.6667  233000.0000  ...  566033.3333  577466.6667  584050.0  non-university town       1.2135
Illinois     Chicago              138400.0000  143633.3333  147866.6667  152133.3333  156933.3333  ...  206066.6667  208200.0000  212000.0  non-university town       1.0828
Pennsylvania Philadelphia          53000.0000   53633.3333   54133.3333   54700.0000   55333.3333  ...  123433.3333  126933.3333  12870

In [134]:
ttest_ind(
    df_home_price_quarter.loc[df_home_price_quarter["label"] == "university town", "price ratio"].dropna(),
    df_home_price_quarter.loc[df_home_price_quarter["label"] == "non-university town", "price ratio"].dropna(),
)

Ttest_indResult(statistic=-2.998032664179151, pvalue=0.002724063704761454)